Κώδικας Ομάδας NLP: Φακίδης Γιώργος, Πέτρος Φίλος, Γιώργος Βιδάκης

Setup Code to Run the Code Below

In [ ]:
pip install evaluate # For WER,CER metrics

In [ ]:
pip install jiwer

In [ ]:
pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.8 MB/s eta 0:00:00


In [2]:
import nltk
from collections import Counter

from nltk.corpus import brown, stopwords,europarl_raw,abc
from nltk import tokenize, sent_tokenize, unique_list, word_tokenize
nltk.download("brown")
nltk.download("europarl_raw")
nltk.download("stopwords")
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download("abc")
from nltk.util import ngrams
from Levenshtein import distance
import string
import numpy as np
import random
from evaluate import load
from math import log2
import math

from nltk.corpus import brown, stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package brown to /home/georgefkd/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package europarl_raw to
[nltk_data]     /home/georgefkd/nltk_data...
[nltk_data]   Package europarl_raw is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/georgefkd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/georgefkd/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/georgefkd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package abc to /home/georgefkd/nltk_data...
[nltk_data]   Package abc is already up-to-date!
/home/georgefkd/programming/aueb-cs-masters/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedoc

(1) Implement (in any programming language) a bigram and a trigram language model for sentences, using Laplace smoothing


In [3]:
brown_corpus_reader = brown.words(categories="news")
brown_corpus_texts = list(map(lambda var_token: var_token.lower(), brown_corpus_reader))

split_index = int(len(brown_corpus_texts) * 0.8)
var_check_increase_index = 4
split_index += var_check_increase_index
print("Splitting index point is:", split_index, "Total length is:", len(brown_corpus_texts))
test_corpus_texts = brown_corpus_texts[split_index:]
training_corpus_texts = brown_corpus_texts[:split_index]

print("Training List: Length with stopwords:", len(training_corpus_texts))
print("Testing List:  Length with stopwords:", len(test_corpus_texts))
# Stopwords δεν χρειάζονται σε κάποιες περιπτώσεις π.χ. θέλει classification
training_corpus_texts = [var_token for var_token in training_corpus_texts if not var_token in stopwords.words("english")]
test_corpus_texts = [var_token for var_token in test_corpus_texts if not var_token in stopwords.words("english")]

print("First tokens are:", brown_corpus_reader[:20])
print("Length of Training Corpus is:", len(training_corpus_texts), "First tokens are:", training_corpus_texts[:20])
print("Length of Testing Corpus is:", len(test_corpus_texts), "First tokens are:", test_corpus_texts[:20])

word_counter = Counter(training_corpus_texts)
print("Length of Counter is:", len(word_counter))
# Filter out words that do appear less than 9 times

MIN_FREQUENCY = 9
word_counter = Counter({var_key: var_value for var_key, var_value in word_counter.items() if var_value > MIN_FREQUENCY})
print("Length of Counter is:", len(word_counter))
print("Most common are:", word_counter.most_common(5))
print("Least common are:", word_counter.most_common()[-5:])

vocab_words = list(word_counter.keys())
print("Length of Vocabulary:", len(vocab_words), "Type:", type(vocab_words))

training_corpus_texts = [var_token if var_token in vocab_words else "<UNK>" for var_token in training_corpus_texts]
print("Length of Corpus with unknown words replaced:", len(training_corpus_texts))

training_text = " ".join(training_corpus_texts)
training_sentences = sent_tokenize(training_text)
testing_text = " ".join(test_corpus_texts)

def calculate_ngrams(training_document):
    sentences_for_training = sent_tokenize(training_document)
    count_unigrams = Counter()
    count_bigrams = Counter()
    count_trigrams = Counter()
    for sent in sentences_for_training:
        count_unigrams.update([var_unigram for var_unigram in ngrams(sent.split(), 1, pad_left = True, pad_right=True, left_pad_symbol ="<start>", right_pad_symbol="<end>")])
        count_bigrams.update([var_bigram for var_bigram in ngrams(sent.split(), 2, pad_left = True, pad_right=True, left_pad_symbol ="<start>", right_pad_symbol="<end>")])
        count_trigrams.update([var_trigram for var_trigram in ngrams(sent.split(), 3, pad_left = True, pad_right=True, left_pad_symbol ="<start>", right_pad_symbol="<end>")])
    return count_unigrams,count_bigrams,count_trigrams
counter_unigram,counter_bigram,counter_trigram = calculate_ngrams(training_text)

# calculate probabilities from frequencies


print("Top 10 from unigram,bigram and trigram model")
print(counter_unigram.most_common(10))
print(counter_bigram.most_common(10))
print(counter_trigram.most_common(10))
print(counter_bigram.most_common()[-10:])

dict_counter_unigrams = {var_key[0]: var_value for var_key, var_value in counter_unigram.items()}
dict_counter_unigrams.update({"<start>": len(training_sentences)})
dict_counter_bigrams = {(var_key[0], var_key[1]): var_value for var_key, var_value in counter_bigram.items()}
dict_counter_bigrams.update({("<start>","<start>"): len(training_sentences)})
dict_counter_trigrams = {(var_key[0], var_key[1], var_key[2]): var_value for var_key, var_value in counter_trigram.items()}

print("keys are:", dict_counter_unigrams.keys())
print("count is:", dict_counter_unigrams['<start>'])


vocabulary_size = len(vocab_words)
print("vocabulary size is:", vocabulary_size)

def calculate_bigram_probabilities(bigram_counters,vocab_size,unigram_counters):
    bigram_probabilities = { }
    for (word1,word2), var_bigram_value in bigram_counters.items():
        #Formula for Bigram Language Model using Laplace Smoothing
        bigram_probabilities.update({(word2, word1): (var_bigram_value + 1) / (unigram_counters[word1] + vocab_size)})
    return bigram_probabilities
dict_bigram_probabilities = calculate_bigram_probabilities(dict_counter_bigrams,vocabulary_size,dict_counter_unigrams)

def calculate_trigram_probabilities(trigram_counters,vocab_size,bigram_counters):
    trigram_probabilities = { }
    for (word1,word2,word3), trigram_value in trigram_counters.items():
        #Formula for Trigram Language Model using Laplace Smoothing
        trigram_probabilities.update({(word3, word1, word2): (trigram_value + 1) / (bigram_counters[(word1, word2)] + vocab_size)})
    return trigram_probabilities

dict_trigram_probabilities = calculate_trigram_probabilities(dict_counter_trigrams,vocabulary_size,dict_counter_bigrams)



# print(dict_bigram_probabilities)
print(dict_trigram_probabilities)

Splitting index point is: 80447 Total length is: 100554
Training List: Length with stopwords: 80447
Testing List:  Length with stopwords: 20107
First tokens are: ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that']
Length of Training Corpus is: 51217 First tokens are: ['fulton', 'county', 'grand', 'jury', 'said', 'friday', 'investigation', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'evidence', "''", 'irregularities', 'took', 'place', '.', 'jury']
Length of Testing Corpus is: 12522 First tokens are: ['show', 'self-restraint', ',', 'get', 'orders', '.', 'circumstances', ',', 'protection', 'relatively', 'small', 'manufacturers', 'engage', 'exactly', 'kind', 'conspiracy', 'giants', 'latter', 'convicted', '.']
Length of Counter is: 11339
Length of Counter is: 881
Most common are: [(',', 4048), ('.', 3234), ('``', 602), ("''", 580), ('sai

(2) Estimate the language cross-entropy and perplexity of your two models on a test subset of
the corpus, treating the entire test subset as a single sequence of **sentences**

In [4]:
# Cross Entropy Formulas
subset_sentences = sent_tokenize(testing_text)[:10]

def calculate_bigram_cross_entropy(sentences,bigram_probabilities,counter_unigrams,vocab_size):
    bigram_sum_log_likelihood = 0
    total_tokens = 0
    for sent in sentences:
        bigram_sentence = "<start> " + sent + " <end>"
        
        sentence_tokens = bigram_sentence.split()
        total_tokens += len(sentence_tokens)
        for bigram_token in range(1, len(sentence_tokens)-1):
            # print("token is:", sentence_tokens[var_bigram_token])
            # print((sentence_tokens[var_bigram_token+1], sentence_tokens[var_bigram_token]))
            bigram = (sentence_tokens[bigram_token+1], sentence_tokens[bigram_token])
            bigram_sum_log_likelihood += math.log2(bigram_probabilities.get(bigram, 1 / (counter_unigrams.get(bigram_token, 0.0) + vocab_size)))
    cross_entropy = - bigram_sum_log_likelihood / total_tokens
    perplexity_ = math.pow(2, cross_entropy)
    return cross_entropy,perplexity_

def calculate_trigram_cross_entropy(sentences,trigram_probabilities,counter_bigrams,vocab_size):
    trigram_sum_log_likelihood = 0
    total_tokens = 0
    for sent in sentences:
        sent_trigram = "<start1> <start2> " + sent + " <end>"
        sentence_tokens = sent_trigram.split()
        total_tokens += len(sentence_tokens)
        for trigram_token in range(2, len(sentence_tokens)-2):
            bigram = (sentence_tokens[trigram_token+1], sentence_tokens[trigram_token])
            trigram = (sentence_tokens[trigram_token+2], sentence_tokens[trigram_token], sentence_tokens[trigram_token+1])
            trigram_sum_log_likelihood += math.log2(trigram_probabilities.get(trigram, 1 / (counter_bigrams.get(bigram, 0.0) + vocab_size)))
    cross_entropy = - trigram_sum_log_likelihood / total_tokens
    perplexity_ = math.pow(2, cross_entropy)
    return cross_entropy,perplexity_
bigram_cross_entropy,bigram_perplexity = calculate_bigram_cross_entropy(subset_sentences,dict_bigram_probabilities,counter_unigram,vocabulary_size)
trigram_cross_entropy,trigram_perplexity = calculate_trigram_cross_entropy(subset_sentences,dict_trigram_probabilities,counter_bigram,vocabulary_size)
print("Bigram: Cross Entropy is:", bigram_cross_entropy, "Perplexity is:", bigram_perplexity)
print("Trigram: Cross Entropy is:", trigram_cross_entropy, "Perplexity is:", trigram_perplexity)


Bigram: Cross Entropy is: 7.600183531929075 Perplexity is: 194.03640321432016
Trigram: Cross Entropy is: 6.999998573768174 Perplexity is: 127.99987346072571


In [7]:
subset_complete_sentences = sent_tokenize(training_text)[:2]
print("Sentence 1:", subset_complete_sentences[0], "Length is:", len(subset_complete_sentences[0].split()))
print("Sentence 2:", subset_complete_sentences[1], "Length is:", len(subset_complete_sentences[1].split()))

def bigram_find_top_2_autocompl(last_word_in_sentence,bigram_probs,unigrams_counter):
    max_bigram_probability = 0.0
    max_bigram_word = " "
    max2_bigram_probability = 0.0
    max2_bigram_word = " "
    for word in vocab_words:
        bigram_tuple = (word, last_word_in_sentence)
        bigram_probabilty = bigram_probs.get(bigram_tuple, 1 / (unigrams_counter.get(bigram_tuple[1], 0.0) + vocabulary_size))
        if  max_bigram_probability < bigram_probabilty:
            # print("index", index, "new candidate bigram word:", word, "tuple: ", bigram_tuple)
            max2_bigram_probability = max_bigram_probability
            max2_bigram_word = max_bigram_word
            max_bigram_probabilty = bigram_probabilty
            max_bigram_word = word
        elif max2_bigram_probability < bigram_probabilty:
            max2_bigram_probability = bigram_probabilty
            max2_bigram_word = word
    return max_bigram_probability,max_bigram_word,max2_bigram_probability,max2_bigram_word
def trigram_find_top_2_autocompl(last_word_in_sentence,before_last_word_in_sentence,trigram_probs,bigrams_counter):
    max_trigram_probability = 0.0
    max_trigram_word = " "
    max2_trigram_probability = 0.0
    max2_trigram_word = " "
    for word in vocab_words:
        trigram_tuple = (word, before_last_word_in_sentence, last_word_in_sentence)
        trigram_probability = trigram_probs.get(trigram_tuple, 1 / (bigrams_counter.get((trigram_tuple[1], trigram_tuple[2]), 0.0) + vocabulary_size))
        if max_trigram_probability < trigram_probability:
            # print("index", index, "new candidate trigram word:", word, "tuple: ", bigram_tuple)
            max2_trigram_probability = max_trigram_probability
            max2_trigram_word = max_trigram_word
            max_trigram_probability = trigram_probability
            max_trigram_word = word
        elif max2_trigram_probability < trigram_probability:
            max2_trigram_probability = trigram_probability
            max2_trigram_word = word
    return max_trigram_probability,max_trigram_word,max2_trigram_probability,max2_trigram_word


def split_sentence_in_half(sent,for_bigram):
    words_of_sentence = sent.split()
    total_tokens = len(words)
    max_unremoved_tokens = total_tokens // 2 + 1
    if for_bigram:
        return ["<start>"] + words_of_sentence[:max_unremoved_tokens]
    else:
        return ["<start1>", "<start2>"] + words_of_sentence[:max_unremoved_tokens]

def should_ignore(w):
    return w in [",",".","<UNK>","-"]

for sentence in subset_complete_sentences:
    words = sentence.split()
    total_tokens = len(sentence.split())
    max_unremoved_tokens = total_tokens // 2 + 1
    incomplete_bigram_sentence_words = split_sentence_in_half(sentence,False)
    incomplete_trigram_sentence_words = split_sentence_in_half(sentence,True)
    print("Bigram Sentence Incomplete:", incomplete_bigram_sentence_words)
    print("Trigram Sentence Incomplete:", incomplete_trigram_sentence_words)
    current = max_unremoved_tokens + 1
    while current < total_tokens - 2:
    # for index in range(total_tokens - max_unremoved_tokens + 1):    
        # bigram_prev_last_word = incomplete_bigram_sentence_words[index - 2]
        bigram_last_word = incomplete_bigram_sentence_words[current - 1] if not(should_ignore(incomplete_bigram_sentence_words[current - 1])) else incomplete_bigram_sentence_words[current - 2]
        print(f"Bigram Last Word: {bigram_last_word}")
        max_bigram_probability,max_bigram_word,max2_bigram_probability,max2_bigram_word = bigram_find_top_2_autocompl(bigram_last_word,dict_bigram_probabilities,dict_counter_unigrams) 
        
        trigram_prev_last_word = incomplete_trigram_sentence_words[current-2] if not(should_ignore(incomplete_trigram_sentence_words[current-2])) else incomplete_trigram_sentence_words[current-3]
        trigram_last_word = incomplete_trigram_sentence_words[current - 2]
        print(f"Trigram previous last and last words: {trigram_prev_last_word}-{trigram_last_word}")
        max_trigram_probability,max_trigram_word,max2_trigram_probability,max2_trigram_word = trigram_find_top_2_autocompl(trigram_last_word,trigram_prev_last_word,dict_trigram_probabilities,dict_counter_bigrams)
            
        incomplete_bigram_sentence_words.append(max_bigram_word)
        incomplete_trigram_sentence_words.append(max_trigram_word)
        current = current + 1
    print("Bigram Complete:", incomplete_bigram_sentence_words)
    print("Trigram Complete:", incomplete_trigram_sentence_words)

Sentence 1: fulton county grand jury said friday investigation <UNK> recent primary election <UNK> `` evidence '' <UNK> took place . Length is: 19
Sentence 2: jury said <UNK> <UNK> city executive committee , <UNK> charge election , `` <UNK> <UNK> <UNK> city atlanta '' <UNK> election <UNK> . Length is: 23
Bigram Sentence Incomplete: ['<start1>', '<start2>', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'investigation', '<UNK>', 'recent', 'primary']
Trigram Sentence Incomplete: ['<start>', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'investigation', '<UNK>', 'recent', 'primary']
Bigram Last Word: recent
Trigram previous last and last words: recent-recent
Bigram Last Word: primary
Trigram previous last and last words: primary-primary
Bigram Last Word: library
Trigram previous last and last words: fulton-fulton
Bigram Last Word: library
Trigram previous last and last words: fulton-fulton
Bigram Last Word: library
Trigram previous last and last words: fulton-fulton
Bigra

(3) Write some code to show how your bigram and trigram language models can auto-
complete an incomplete sentenc

(4),(5),(6)

Implement a context-aware Spelling corrector, generate artifical

datasets for it, and evaluate it using WER and CER

In [9]:
class SentenceCorrectionResult:
    initial_sentence = ""
    typos_sentence = ""
    mistake_probability = 0.0
    corrected_sentence = ""
    wer = -1
    cer = -1
    lev_weight = -1
    lm_weight = -1
    def append_to_file(self,filepath):
        with open(filepath, mode='a', encoding='utf-8') as file:
            file.write(f"{self.initial_sentence},{self.typos_sentence},{self.corrected_sentence},{self.mistake_probability},{self.wer},{self.cer},{self.lev_weight},{self.lm_weight}\n")

### Context-Aware Spelling Corrector

class AutoCorrector:
    _texts = [] #= [("","")]
    # this makes it so that we can use anything where we can do .get((w1,w2))
    # to get the probability
    _bigram_model = dict()
    _trigram_model = dict()
    _file_output = "Per-Sentence-Results-Final.csv"

    _sentences_to_evaluate_on = 5
    _mistake_probability = 0.18

    _beam_search_max_depth = 7
    _beam_search_beam_width = 4
    _beam_search_candidates_amount = 10
    _beam_search_use_bigram = False
    _beam_search_weights_for_prob = (0.65, 0.35)


    _sentences = []
    _sentences_with_typos = []

    _vocabulary = []
    def run_with_default_settings(self,bigram_model,trigram_model,vocab):
        self._bigram_model = bigram_model
        self._trigram_model = trigram_model
        self._vocabulary = vocab
        assert (len(self._vocabulary) > 0)
        assert (self._beam_search_candidates_amount > 0)

        self.get_default_texts()
        self.get_random_sentences_from_texts()
        self.add_typos_to_sentences()
        self.correct_sentences()

    # (v) Create an artificial test dataset to evaluate your context-aware spelling corrector
    def get_default_texts(self):
        self._texts.append((europarl_raw.english.raw(),"Sample European Parliament Proceedings Parallel Corpus"))
    def get_random_sentences_from_texts(self):
        assert (len(self._texts) > 0)
        text,title = random.choice(self._texts)
        sentences = sent_tokenize(text)
        random_sentences = [random.choice(sentences) for _ in range(self._sentences_to_evaluate_on)]
        self._sentences = [s.replace("\n"," ").replace(","," ").lower() for s in random_sentences]
    def add_typos_to_sentences(self):
        self._sentences_with_typos = list(map(lambda sentence: self.add_typos_to_sentence(sentence),self._sentences))

    ### (VI) Evaluate your context-aware spelling corrector in terms of
    # Word Error Rate (WER) and
    # Character Error Rate (CER).
    def correct_sentences(self):
        assert (len(self._sentences) == len(self._sentences_with_typos))
        print("========Correcting sentences=========")
        print("\n")
        with open(self._file_output, mode="a", encoding='utf-8') as file:
            file.write(
                "Initial Sentence,Sentence with Typos,Corrected Sentence,Mistake Probability,WER,CER,Levenshtein Weight,Language Model Weight\n")
        weight_options = [
            #LM λ1,LEV λ2
            (0.0, 1.0),
            (0.1, 0.9),
            (0.2, 0.8),
            (0.3, 0.7),
            (0.4, 0.6),
            (0.5, 0.5),
            (0.6, 0.4),
            (0.7, 0.3),
            (0.8, 0.2),
            (0.9, 0.1),
            (1.0, 0.0),
        ]
        for weights in weight_options:
            self._beam_search_weights_for_prob = weights
            avg_word_error_rate = 0
            avg_character_error_rate = 0
            for index in range(len(self._sentences)):
                correct = self._sentences[index]
                with_typos = self._sentences_with_typos[index]
                corrected = self.correct_sentence(with_typos)

                wer = AutoCorrector.WER([corrected], [correct])
                cer = AutoCorrector.CER([corrected], [correct])

                # print("Initial Sentence->", correct)
                # print("With added typos->", with_typos)
                # print("Final correct sentence->", corrected)
                print("WER->", wer)
                print("CER->", cer)

                res = SentenceCorrectionResult()
                res.initial_sentence = correct
                res.typos_sentence = with_typos
                res.mistake_probability = self._mistake_probability
                res.corrected_sentence = corrected
                res.wer = wer
                res.cer = cer
                res.lm_weight = self._beam_search_weights_for_prob[0]
                res.lev_weight = self._beam_search_weights_for_prob[1]
                res.append_to_file(self._file_output)

                avg_word_error_rate = avg_word_error_rate + wer
                avg_character_error_rate = avg_character_error_rate + cer
            avg_word_error_rate = avg_word_error_rate / len(self._sentences)
            avg_character_error_rate = avg_character_error_rate / len(self._sentences)
            print(f"For weights(lm,lev): {self._beam_search_weights_for_prob}")
            print("For mistake probability: ", self._mistake_probability)
            print("Our model's AVG WER is: ", avg_word_error_rate)
            print("Our model's AVG CER is: ", avg_character_error_rate)
    def correct_sentence(self, with_typos):
        # have not yet added <start>, <end>
        initial_state = word_tokenize(with_typos)
        initial_state = list(map(lambda word: word.lower(), initial_state))
        # initial_state.insert(0,"<start>")
        # initial_state.append("<end>")
        best_correction = self.beam_search_decode(
            initial_state,
            self._beam_search_max_depth,
            self._beam_search_beam_width,
            self.get_candidate_corrections_with_levenshtein,
            self.score)

        return best_correction

    def add_typos_to_sentence(self,complete_text):
        final_text = ""
        def is_non_space_char(ch):
            return ch == ' ' or ch == '\t' or ch == '\r' or ch == '\n'
        for i in range(len(complete_text)):
            character = complete_text[i]
            if is_non_space_char(character):
                # so we can maintain the same sentence format
                final_text += character
                continue
            p = np.random.uniform(0, 1)
            if p <= self._mistake_probability:
                final_text = final_text + AutoCorrector.generate_random_character(character)  # random character
            else:
                final_text = final_text + character
        return final_text

    @staticmethod
    def generate_random_character(ch):
        # when a user does a typo he most likely was using the correct case
        # but this is too linguistic for this case
        if ch.islower():
            return random.choice(string.ascii_lowercase)
        else:
            return random.choice(string.ascii_uppercase)


    def get_n_closest_words(self,word,n):
        # print("Word: ",word)
        # assert(type(word) == str)
        if word == "?" or word == "," or word == ".":
            return [(word,1.0) for _ in range(n)]

        def edit_distance(word1):
            return distance(word1, word)
        closest_words = sorted(self._vocabulary, key=edit_distance)[:n]
        # something is going wrong here
        probs = []
        for close_word in closest_words:
            dist = edit_distance(close_word)
            if dist == 0:
                probs.append(1)
            else:
                probs.append(1.0/(dist + 1))
        return list(zip(closest_words, probs))
    def get_candidate_corrections_with_levenshtein(self, sentence_prob):
        assert (len(sentence_prob[0]) > 0)
        word_alternatives = []
        for word in sentence_prob[0]:
            if type(word) != str :continue
            close_vocab_words = self.get_n_closest_words(word,7)
            word_alternatives.append(close_vocab_words)
        candidates = []
        for i in range(self._beam_search_candidates_amount):
            sentence = []
            for words_to_choose_from in word_alternatives:
                word_and_prob = max(words_to_choose_from,key=lambda x: x[1])
                sentence.append(word_and_prob)
            probability_of_sentence = 1
            result = []
            for word_prob in sentence:
                probability_of_sentence *= word_prob[1]
                result.append(word_prob[0])
            candidates.append((result,probability_of_sentence))
        return candidates


    def score(self,state):
        # Calculate the probability of the word sequence using the bigram model
        # or trigram model and the levenshtein distance(of the last word of state)
        # State-> Tuple(Sentence->Words[],Lev_Prob_Of_sentence))
        sentence = state[0]
        lev_probability_of_sentence = state[1]
        lm_probability = 1.0
        if self._beam_search_use_bigram:
            for i in range(1, len(sentence)):
                prev_word, word = sentence[i - 1], sentence[i]
                lm_probability = lm_probability * self._bigram_model.get((word,prev_word), 0.0)
        else:
            for i in range(2, len(sentence)):
                prev_prev_word,prev_word,word = sentence[i-2],sentence[i - 1], sentence[i]
                lm_probability = lm_probability * self._trigram_model.get((word,prev_prev_word, prev_word), 0.0)
        #-l1*log(P(w1|t1))-l2*log(P(t1_k))-> Slide 18 Part_01
        result = self._beam_search_weights_for_prob[1] * log2(lev_probability_of_sentence)
        if lm_probability != 0:
           result += self._beam_search_weights_for_prob[0] * lm_probability
        return -result

    def is_out_of_vocabulary(self,word):
        return not (word in self._vocabulary)
    @staticmethod
    def WER(predictions, references):
        wer = load("wer")
        return wer.compute(predictions=predictions, references=references)
    @staticmethod
    def CER(predictions, references):
        cer = load("cer")
        return cer.compute(predictions=predictions, references=references)

    def beam_search_decode(self,initial_state, max_depth, beam_width, generate_candidates_fn, score_fn):
        candidates = [(initial_state, 1.0)]
        print("========BEAM SEARCH=======")
        print("For sentence: ",initial_state)
        for depth in range(max_depth):
            new_candidates = []
            for candidate, prob in candidates:
                for next_state in generate_candidates_fn((candidate,prob)):
                    new_prob = prob * score_fn(next_state)
                    new_candidates.append((next_state[0],new_prob*next_state[1]))

            # print('\n***** NEW candidates *****')
            #print(new_candidates)
            new_candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)
            # print('***** Sorted')
            #print(new_candidates)
            #print(f'***** Chosen candidates (top-{beam_width})')
            candidates = new_candidates[:beam_width]
            #print(candidates)
        assert (len(candidates) > 0)
        best_sequence, best_prob = max(candidates, key=lambda x: x[1])
        return " ".join(best_sequence)


corrector = AutoCorrector()
corrector.run_with_default_settings(dict_bigram_probabilities, dict_trigram_probabilities, vocab_words)

========Correcting sentences=========


========BEAM SEARCH=======
For sentence:  ['thh', 'amoumt', 'quoted', 'ii', 'dmnnhmeno', 'no', '12', 'ddes', 'got', 'howrvyg', 'tallz', 'yith', 'the', 'budget', 'sew', 'on', 'thq', 'iasis', 'of', 'agenda', 'r0aa', '.']
WER-> 0.7727272727272727
CER-> 0.3577981651376147
========BEAM SEARCH=======
For sentence:  ['it', 'is', 'also', 'valid', 'flr', 'certynn', 'rzgions', 'haadicaqped', 'by', 'their', 'isolation', 'dun', 'to', 'mcunuains', 'fjr', 'eqaaple', 'k']
WER-> 0.8235294117647058
CER-> 0.29591836734693877
========BEAM SEARCH=======
For sentence:  ['i', 'world', 'ask', 'that', 'thiy', 'reconsidel', 'hince', 'thij', 'is', 'not', 'the', 'case', 'x']
WER-> 0.7692307692307693
CER-> 0.32786885245901637
========BEAM SEARCH=======
For sentence:  ['in', 'any', 'dvent', 'i', 'look', 'fnrwacd', 'to', 'the', 'documeni', 'which', 'oas', 'approvqd', 'wy', 'thv', 'coelisbion', 'todaq', '.']
WER-> 0.7058823529411765
CER-> 0.2840909090909091
========BEAM SEARCH